<a href="https://colab.research.google.com/github/cristianbossolasco/scraper-agencias-viajes/blob/main/scraper_busplus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

Executing: /tmp/apt-key-gpghome.6YevrOv5m5/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.3tvGFfSEBq/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.FStyUtS0Uf/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Ign:2 https://developer.download.nvidia

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

In [20]:
def get_data(origen_parada, destino_parada, fecha_ida):
  url = f'https://checkout.busplus.com.ar/servicios?origen_parada={origen_parada}&destino_parada={destino_parada}&fecha_ida={fecha_ida}&fecha_vuelta=&pasajeros=1&empresa=&cupondescuento='

  options = Options()
  options.add_argument("--headless")
  options.add_argument("--no-sandbox")

  options.headless = True

  driver = webdriver.Chrome("/usr/bin/chromedriver", options=options)

  driver.get(url)

  # get_attribute('innerHTML') para obtener el html

  items = []
  filas  = driver.find_elements(By.CLASS_NAME,"fila_servicio.servicio_fila.idatr")
  for fila in filas:
    if not "hide " in fila.get_attribute(name="class"):
      id = fila.get_attribute('id')

      empresa_img = fila.find_element(By.TAG_NAME,"td").find_element(By.TAG_NAME, 'img').get_attribute('src')
      salida = driver.find_element(By.XPATH, f'//*[@id="{id}"]/td[2]').text.replace('\n', ' ')
      llegada = driver.find_element(By.XPATH, f'//*[@id="{id}"]/td[4]').text.replace('\n', ' ')
      escala = driver.find_element(By.XPATH, f'//*[@id="{id}"]/td[5]/a').text.replace('\n', ' ')
      duracion = driver.find_element(By.XPATH, f'//*[@id="{id}"]/td[6]').text.replace('\n', ' ')
      categoria = driver.find_element(By.XPATH, f'//*[@id="{id}"]/td[7]/ul/li[1]').text
      butacas_libres = driver.find_element(By.XPATH, f'//*[@id="{id}"]/td[8]/ul/li[1]').text
      precio = driver.find_element(By.XPATH, f'//*[@id="{id}"]/td[9]/ul/li[1]/span').text


      item_json = {
          "origen_parada": origen_parada,
          "destino_parada": destino_parada,
          "fecha_ida": fecha_ida,
          "empresa_img": empresa_img,
          "salida": salida,
          "llegada": llegada,
          "escala": escala,
          "duracion": duracion,
          "categoria": categoria,
          "butacas_libres": butacas_libres,
          "precio": precio
      }

      items.append(item_json)

  driver.quit()
  return items


In [21]:
get_data(28, 176, '2023/01/29')

<ipython-input-20-3b2e7f1dc73b>:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("/usr/bin/chromedriver", options=options)


[{'origen_parada': 28,
  'destino_parada': 176,
  'fecha_ida': '2023/01/29',
  'empresa_img': 'https://checkout.busplus.com.ar/images/v2/empresas/50.png',
  'salida': '11:15 Dom 29/01',
  'llegada': '18:10 Dom 29/01',
  'escala': 'Directo',
  'duracion': '06:55hs',
  'categoria': 'Estandar',
  'butacas_libres': '29',
  'precio': '$5400'},
 {'origen_parada': 28,
  'destino_parada': 176,
  'fecha_ida': '2023/01/29',
  'empresa_img': 'https://checkout.busplus.com.ar/images/v2/empresas/50.png',
  'salida': '16:40 Dom 29/01',
  'llegada': '23:35 Dom 29/01',
  'escala': 'Directo',
  'duracion': '06:55hs',
  'categoria': 'Semi Cama',
  'butacas_libres': '5',
  'precio': '$6250'},
 {'origen_parada': 28,
  'destino_parada': 176,
  'fecha_ida': '2023/01/29',
  'empresa_img': 'https://checkout.busplus.com.ar/images/v2/empresas/78.png',
  'salida': '08:30 Dom 29/01',
  'llegada': '18:10 Dom 29/01',
  'escala': 'Transbordo',
  'duracion': '09:40hs',
  'categoria': 'Cama Ejecutivo',
  'butacas_libre